In [1]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.datasets import cifar10
(x_train, y_train),(x_test, y_test) = cifar10.load_data()

2023-03-07 12:46:41.860809: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
x_train = x_train / 255
x_test = x_test / 255

from tensorflow.keras import utils

In [3]:
y_train[0],y_train[1],y_train[2]

(array([6], dtype=uint8), array([9], dtype=uint8), array([9], dtype=uint8))

In [4]:
y_train = utils.to_categorical(y_train)
y_train[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [5]:
y_test = utils.to_categorical(y_test)

In [6]:
y_test[0]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [7]:
#함수형 방식 , functional 방식
from tensorflow.keras.layers import Input
input_img = Input(shape = (32,32,3) ,name='input_start')

In [8]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D
tower_0 = Conv2D(64, (1,1) , padding='same',activation='relu',name = 'tower_0_start')(input_img)
####

tower_1 = Conv2D(64, (1,1) , padding='same',activation='relu', name = 'tower_1_start')(input_img)
tower_1 = Conv2D(64, (3,3) , padding='same',activation='relu')(tower_1)

###

tower_2 = MaxPooling2D((3,3) , strides=(1,1), padding='same')(input_img)
tower_2 = Conv2D(64, (3,3) , padding='same',activation='relu')(tower_2)


tower_3 = Conv2D(64, (1,1) , padding='same',activation='relu')(input_img)
tower_3 = Conv2D(64, (3,3) , padding='same',activation='relu')(tower_3)
tower_3 = Conv2D(64, (3,3) , padding='same',activation='relu')(tower_3)
##

2023-03-07 12:47:21.056949: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
output = tf.keras.layers.concatenate([tower_0,tower_1,tower_2,
                                      tower_3], axis = 3)

In [10]:
from tensorflow.keras.layers import Flatten, Dense
output = Flatten()(output)
out = Dense(10, activation = 'softmax')(output)

from tensorflow.keras.models import Model
model = Model(inputs = input_img, outputs = out)

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_start (InputLayer)       [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_2 (Conv2D)              (None, 32, 32, 64)   256         ['input_start[0][0]']            
                                                                                                  
 tower_1_start (Conv2D)         (None, 32, 32, 64)   256         ['input_start[0][0]']            
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 32, 32, 3)    0           ['input_start[0][0]']            
                                                                                              

In [11]:
from tensorflow.keras.optimizers import SGD
momentum_param = 0.9
lrate = 0.1
sgd = SGD(learning_rate = lrate, momentum = momentum_param)

In [12]:
model.compile(loss= 'categorical_crossentropy', optimizer = sgd)

In [13]:
model.fit(x_train, y_train, 
          validation_data= (x_test,y_test),
          epochs=10, batch_size=64)

Epoch 1/10
708/782 [==========================>...] - ETA: 1:25 - loss: 2.1744

KeyboardInterrupt: 